In [1]:
!pip install --upgrade openai python-dotenv scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 1.9 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: openai
    Found existing installation: openai 1.16.2
    Uninstalling openai-1.16.2:
      Successfully uninstalled openai-1.16.2


## Idea
Fetch all symbols for TO exchange from Finnhub.io (free) then use OpenAi to add risks for each ticker.

In [8]:
from dotenv import dotenv_values
import os
import json
from openai import OpenAI
import requests
from tqdm import tqdm

config = dotenv_values(".env")

In [5]:
client = OpenAI(api_key=config["OPEN_API_KEY"])

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system",
         "content": "You are a stock analyst specializing in the analysis of risks and competitive advantages"},
        {"role": "user", "content": "Tell me what I need to know about L.TO"}
    ]
)

In [6]:
completion.choices[0].message

ChatCompletionMessage(content="L.TO is the stock symbol for Loblaw Companies Limited, a major Canadian food retailer. When analyzing the risks and competitive advantages of Loblaw Companies Limited, there are several key factors to consider:\n\n**Risks:**\n1. **Competition**: Loblaw operates in a highly competitive market with other major retailers and discount stores. Increased competition can impact its market share and profitability.\n2. **Regulatory Environment**: Changes in regulations related to food safety, labeling, or labor laws can present risks for Loblaw and impact its operational costs.\n3. **Economic Factors**: Loblaw's business is sensitive to economic conditions that can affect consumer spending habits and overall demand for its products.\n4. **Supply Chain Disruptions**: Any disruptions in the supply chain, whether due to natural disasters, trade issues, or other factors, can impact Loblaw's ability to meet consumer demand and affect its financial performance.\n5. **On

In [9]:
# Using Finnhub to fetch all symbols
# this endpoint is free, but requires registration
data = requests.get(f"https://finnhub.io/api/v1/stock/symbol?exchange=TO&token={config['FINNHUB_TOKEN']}")

In [18]:
symbols = data.json()
symbols[0]

{'currency': 'CAD',
 'description': 'VANGUARD US DIVIDEND APPRECI',
 'displaySymbol': 'VGG.TO',
 'figi': 'BBG004Z0DYT2',
 'isin': None,
 'mic': 'XTSE',
 'shareClassFIGI': 'BBG004Z0DYQ5',
 'symbol': 'VGG.TO',
 'symbol2': '',
 'type': 'ETP'}

In [100]:
data_dir = "data/"
result = []

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

for symbol in tqdm(symbols):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system",
             "content": "You are a stock analyst specializing in the analysis of risks and competitive advantages. You will be giving response in json format. JSON output will have the following fields description, competitive advantage, product, and risks. Description will explain what a company does. Competitive advantage is a list of advantages. Product will have one value which tells what's the company is producing. Risks is going to be a list of risks."},
            {"role": "user", "content": f"Tell me what I need to know about '{symbol['description']}'"}
        ]
    )
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    file_path = os.path.join(data_dir, symbol["symbol"])

    data = completion.choices[0].message.content
    # Write your dictionary to a file
    with open(file_path, 'w') as f:
        json.dump(data, f)
        result.append(completion)

100%|██████████| 2600/2600 [1:13:01<00:00,  1.69s/it]


In [74]:
symbols[1]

{'currency': 'CAD',
 'description': 'MF MULTIFAC EMER MER INDE ET',
 'displaySymbol': 'MEME.B.TO',
 'figi': 'BBG00MRS6BB7',
 'isin': None,
 'mic': 'XTSE',
 'shareClassFIGI': 'BBG00MRS6C98',
 'symbol': 'MEME.B.TO',
 'symbol2': '',
 'type': 'ETP'}

### Note:
- I've checked manually for hallucination couple of stocks at random. I haven't noticed any issues, so I assume that the resulting risks don't contain large amounts of hallucinations.